In [41]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import sph_harm
from scipy.integrate import quad, dblquad

import numpy as np
import trimesh as tri
import torch
import open3d as o3d

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import sys
import os

sys.path.insert(0,"../")
import pyssht
sphere = tri.creation.uv_sphere(count=[16, 16])

In [42]:
np.conj(sph_harm(1, 2,0.1,0.1))

(-0.07635745268016919+0.0076612999759450824j)

In [77]:
def sph_func(l,m):
    def res(theta,phi):
        return sph_harm(m,l, theta,phi)
    return res

In [105]:
def coeff(l,m,f):
    return dblquad(lambda theta,phi: f(theta,phi)*np.abs(sph_harm(m,l,theta,phi))*np.sin(theta), 0,2*np.pi, lambda x: 0, lambda x: np.pi)[0]

In [106]:
f_test = lambda theta,phi: np.sin(theta)*np.cos(phi)

In [107]:
coeff(5,2, f_test)

5.457233093817454e-17

In [108]:
x, y ,z = np.array(sphere.vertices[:,0]), np.array(sphere.vertices[:,1]), np.array(sphere.vertices[:,2])
(r, theta, phi) = pyssht.cart_to_spherical(x, y, z)

In [126]:
coeffs = []
def reconstruct(l,f):
    for m in range(-l,l+1,1):
        def mult_func(theta,phi):
            return(coeff(l,m,f)*sph_func(l,m)(theta,phi)) 
        coeffs.append(mult_func)
    
    def add_func(theta,phi):
        return(np.sum(np.array([elt(theta,phi) for elt in coeffs]), axis=0))
    return add_func